<a href="https://colab.research.google.com/github/verevar/big-data-challenge/blob/main/Kitchen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.1.3'
# spark_version = 'spark-3.<version number>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,965 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.l

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-09-12 23:18:23--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.26MB/s    in 0.7s    

2022-09-12 23:18:25 (1.26 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Kitchen").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

In [7]:
 kitchen_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Kitchen_v1_00.tsv.gz"), sep="\t", header=True)

#Show DataFrame
kitchen_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [8]:
kitchen_df = kitchen_df.dropna()

In [9]:
kitchen_df = kitchen_df.dropDuplicates()

In [10]:
#Count the number of records (rows) in the dataset.
kitchen_df.count()

4879961

In [11]:
# Be sure that the DataFrames match in data type and in column name.
from pyspark.sql.types import * 
kitchen_df = kitchen_df.withColumn("customer_id", kitchen_df["customer_id"].cast(IntegerType()))\
                          .withColumn("product_parent", kitchen_df["product_parent"].cast(IntegerType()))\
                          .withColumn("star_rating", kitchen_df["star_rating"].cast(IntegerType()))\
                          .withColumn("helpful_votes", kitchen_df["helpful_votes"].cast(IntegerType()))\
                          .withColumn("total_votes", kitchen_df["total_votes"].cast(IntegerType()))\
                          .withColumn("review_date", kitchen_df["review_date"].cast(DateType()))

In [12]:
kitchen_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [13]:
review_id = kitchen_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id = review_id.dropDuplicates()
review_id = review_id.dropna()
review_id.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R11CE8T0KV1IE5|   18965311|B000QOH3BE|     866453817| 2015-02-22|
|R11KHKHJDAFHJ3|   50103837|B004M6H6KU|     712448686| 2014-08-20|
|R12S1H6UVOS5V2|   24001017|B0049SHD7C|     639030220| 2015-05-14|
|R13LE3YHBB4QU2|   40411607|B00KJ1HHOM|     703536534| 2015-01-13|
|R14BC0DRNOSSYB|   25105319|B000I1V302|     562172940| 2015-02-02|
|R14EH6R2KZTAZ0|   21875358|B0043095WW|     150530338| 2013-06-05|
|R14FR8B60J149I|   50766554|B005I72LMU|     329693531| 2013-01-24|
|R14PVEHRI5QIFP|   21087900|B008EQ1KB0|     443123829| 2013-11-19|
|R155B9OAXQGQJF|   12356909|B002DUCCDS|     371066242| 2014-06-10|
|R15JZHODZ21SBN|   32972565|B004WSJU8O|     889893533| 2014-02-14|
|R16UH5V9H7UBR2|    8653119|B005RRBMR6|     446038419| 2013-08-10|
|R18BP1BXJXRX9H|   43392959|B00EPEALEM|     403561784| 2015-02

In [17]:
review_id.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [14]:
products_df = kitchen_df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df = products_df.dropna()

products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00004W499|George Foreman GG...|
|B00004S9EJ|Cuisinart DLC-8SY...|
|B000T7C32C|Westmark Multi Pu...|
|B00JP9AX6I|Natizo Ultra Shar...|
|B000HMC5MM|HIC 18/8 Stainles...|
|B00O18LHJU|Chillin' Cubes - ...|
|B00COM0AX4|Arctic Chill Sili...|
|B00FDWENCQ|New Star Foodserv...|
|B008YS1YQO|George Foreman Grill|
|B00ABEKGWM|ITI GX8918 36-Pie...|
|B003IPA126|Epicurean Gourmet...|
|B005ZFYMHM|Tupperware Classi...|
|B00GF5RW8M|Delirium Tremens ...|
|B000Q945FG|BIA Cordon Bleu B...|
|B0084DZ55U|Wildkin Double De...|
|B0016OZPEK|Lausanne Flatware...|
|B000F7FXO6|Pink stoneware Sa...|
|B00069TKPG|Gourmet Standard ...|
|B007XEY6AQ|Welcome Home Bran...|
|B00D19SSE2|Ozeri Serafino Do...|
+----------+--------------------+
only showing top 20 rows



In [18]:
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [15]:
customer_id = kitchen_df.groupby("customer_id")\
              .agg({"customer_id": "count"})\
              .withColumnRenamed("count(customer_id)", "customer_count")
customer_id = customer_id.dropDuplicates()
customer_id = customer_id.dropna()

customer_id.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   43783816|             1|
|   12901154|             2|
|   44367888|             1|
|   16477908|             2|
|   50187791|             1|
|   26181316|             4|
|   23901527|             2|
|   20249496|             2|
|   38515795|             5|
|   46891757|            46|
|   24803411|             3|
|   49781768|             3|
|   26893320|             6|
|   16150370|             6|
|   16175511|             1|
|   37448961|             1|
|   45067021|             2|
|   52129256|             3|
|   18693314|             2|
|   29811841|             5|
+-----------+--------------+
only showing top 20 rows



In [19]:
customer_id.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [16]:
columns = ['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine']
vine_table = kitchen_df[[columns]]
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1000CEJZ30MZL|          5|            1|          2|   N|
|R1002JK1OHHXGG|          5|            0|          0|   N|
|R1009JT567AOTT|          5|            0|          0|   N|
|R100JHAIKOMBJ4|          5|            0|          0|   N|
|R100O6XRNI9JAX|          5|            0|          0|   N|
|R1016UE48A9251|          5|            0|          0|   N|
|R101KPEKO5N1W1|          1|            4|          5|   N|
|R101OQXQGSMXHH|          5|            1|          1|   N|
|R102B4GU1FQEEJ|          2|            3|          5|   N|
|R102JNT9WAJUR7|          3|            0|          0|   N|
|R102KC6UDNXSL2|          4|            0|          0|   N|
|R102PBYHO9GEWG|          1|            1|          1|   N|
|R102WONDOPEYS4|          5|            0|          0|   N|
|R102XXBV8QY381|          5|            

In [20]:
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)

